In [3]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from glob import glob
import os
import rasterio as rio
from PIL import Image
import pandas as pd
import zarr
# import tensorflow_datasets as tfds

In [4]:
print(tf.__version__)

2.4.1


In [5]:
def add_trailing_slash(path):
    if path[-1] != '/':
        path += '/'
    return path

In [6]:
def create_dir(output_dir):
    # If the output folder doesn't exist, create it
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

# Converting to Integer Data Type

In [118]:
def convert_rast(input_dir,output_dir,band_list=[6,7,8],data_type="uint8"):
    
    input_dir = add_trailing_slash(input_dir)
    output_dir = add_trailing_slash(output_dir)
    
    create_dir(output_dir)
    
    file_list = glob(input_dir + "/*/*.tif")
   
    
    for image_path in file_list:
        new_dir = output_dir + image_path.split("/")[-2]
        new_dir = add_trailing_slash(new_dir)
        create_dir(new_dir)
        
        file_path_append = ("/").join(image_path.split("/")[-2:]) 
        
        with rio.open(image_path, 'r') as r:
            rast = r.read(band_list)
            rast = rast.astype(np.uint8)
            r_meta = r.meta
            r_meta["count"] = len(band_list)
            r_meta["dtype"] = data_type


        with rio.open(output_dir + file_path_append, 'w', **r_meta) as outf:
            outf.write(rast)

In [119]:
input_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/test/blah_temp"
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/test/blah_jpg"

convert_rast(input_dir,output_dir)

In [125]:
def convert_to_jpg(input_dir,output_dir,child_dir = True):
    
    input_dir = add_trailing_slash(input_dir)
    output_dir = add_trailing_slash(output_dir)
    
    create_dir(output_dir)
    
    file_list = glob(input_dir + "/*/*.tif") 
    
    
    if child_dir:
    
        for image_path in file_list:

            new_dir = output_dir + image_path.split("/")[-2]

            new_dir = add_trailing_slash(new_dir)

            create_dir(new_dir)

            file_name = image_path.split("/")[-1].split(".")[-2]
            
#             print(file_name)

            im = Image.open(image_path)
            im.thumbnail(im.size)
            im.save(new_dir + file_name + ".jpg", "JPEG", quality=100)

In [126]:
input_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/test/blah_jpg"
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/test/blah_jpg_2"

convert_to_jpg(input_dir,output_dir)

# Creating Dataset from NP Array

In [7]:
path = '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/yes/'

In [8]:
def create_tf_dataset(path=path,chip_size=100,out_zarr_path = 'example.zarr'):
    
    z1 = zarr.open_array(out_zarr_path, mode='w', shape=(47451,9,100,100),
                      dtype='float', fill_value=0)
    
    tif_list = glob(f'{path}/*/{chip_size}/*/*.tif')
    
    for raster_index,tif_path in enumerate(tif_list):
        
        print(f'Adding raster {raster_index + 1} of {len(tif_list)}', end="\r", flush=True)
        
        label = tif_path.split("/")[-4]
        
        with rio.open(tif_path, 'r') as r:
            rast = r.read()
            z1[raster_index] = rast
    
            
    return z1
        
        
    

In [9]:
z1 = create_tf_dataset()



KeyboardInterrupt: 

In [203]:
z1[0][8].max()

0.9079583287239075

In [197]:
r[8].max()

0.9079583

In [184]:
np_describe(r)

 band 1:
              count    100.000000
mean     276.079987
std       30.400253
min      212.000000
25%      257.000000
50%      271.000000
75%      293.750000
max      373.000000
Name: 0, dtype: float64
             
 band 2:
              count    100.000000
mean     440.809998
std       59.781536
min      303.000000
25%      400.500000
50%      439.000000
75%      473.000000
max      620.000000
Name: 0, dtype: float64
             
 band 3:
              count    100.000000
mean     265.130005
std       66.175102
min      176.000000
25%      225.750000
50%      244.000000
75%      296.250000
max      499.000000
Name: 0, dtype: float64
             
 band 4:
              count     100.000000
mean     2444.080078
std       258.430206
min      1748.000000
25%      2300.500000
50%      2392.000000
75%      2635.500000
max      3060.000000
Name: 0, dtype: float64
             
 band 5:
              count     100.000000
mean     2781.929932
std       206.534378
min      2318.000000
2

In [169]:
z1 = zarr.open_array('example.zarr', mode='r')

In [180]:
np_describe(arr[0][8])

 band 1:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: 0, dtype: float64
             
 band 2:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: 0, dtype: float64
             
 band 3:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: 0, dtype: float64
             
 band 4:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: 0, dtype: float64
             
 band 5:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: 0, dtype: float64
             
 band 6:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0

In [174]:
np_describe(z1[8])

 band 1:
              count    100.000000
mean     228.690000
std       22.991301
min      166.000000
25%      214.000000
50%      230.500000
75%      244.000000
max      285.000000
Name: 0, dtype: float64
             
 band 2:
              count    100.000000
mean     331.270000
std       47.087036
min      247.000000
25%      296.250000
50%      335.500000
75%      364.000000
max      466.000000
Name: 0, dtype: float64
             
 band 3:
              count    100.000000
mean     181.650000
std       26.321008
min      128.000000
25%      159.000000
50%      184.000000
75%      197.000000
max      239.000000
Name: 0, dtype: float64
             
 band 4:
              count     100.000000
mean     2407.640000
std       376.743986
min      1688.000000
25%      2170.000000
50%      2422.000000
75%      2613.500000
max      3408.000000
Name: 0, dtype: float64
             
 band 5:
              count     100.000000
mean     2748.360000
std       277.432948
min      2209.000000
2

In [144]:
z1[1] = arr[1]

In [161]:
np.testing.assert_array_equal(z1[100], arr[1])

AssertionError: 
Arrays are not equal

Mismatched elements: 90000 / 90000 (100%)
Max absolute difference: 4026.
Max relative difference: 1.
 x: array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],...
 y: array([[[245.      , 228.      , 226.      , ..., 258.      ,
         236.      , 252.      ],
        [237.      , 221.      , 216.      , ..., 263.      ,...

In [163]:
np_describe(z1[100])

 band 1:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: 0, dtype: float64
             
 band 2:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: 0, dtype: float64
             
 band 3:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: 0, dtype: float64
             
 band 4:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: 0, dtype: float64
             
 band 5:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: 0, dtype: float64
             
 band 6:
              count    100.0
mean       0.0
std        0.0
min        0.0
25%        0

In [162]:
# Describe analysis of each band

def np_describe(arr):

    for i in range(arr.shape[0]):
        print(f""" band {i+1}:
              {pd.DataFrame(arr[i])[0].describe()}
             """)

In [120]:
np_describe(z1[0])

 band 1:
              count    100.000000
mean     276.079987
std       30.400253
min      212.000000
25%      257.000000
50%      271.000000
75%      293.750000
max      373.000000
Name: 0, dtype: float64
             
 band 2:
              count    100.000000
mean     440.809998
std       59.781536
min      303.000000
25%      400.500000
50%      439.000000
75%      473.000000
max      620.000000
Name: 0, dtype: float64
             
 band 3:
              count    100.000000
mean     265.130005
std       66.175102
min      176.000000
25%      225.750000
50%      244.000000
75%      296.250000
max      499.000000
Name: 0, dtype: float64
             
 band 4:
              count     100.000000
mean     2444.080078
std       258.430206
min      1748.000000
25%      2300.500000
50%      2392.000000
75%      2635.500000
max      3060.000000
Name: 0, dtype: float64
             
 band 5:
              count     100.000000
mean     2781.929932
std       206.534378
min      2318.000000
2

In [119]:
np_describe(z1[4751])

 band 1:
              count    100.000000
mean     276.079987
std       30.400253
min      212.000000
25%      257.000000
50%      271.000000
75%      293.750000
max      373.000000
Name: 0, dtype: float64
             
 band 2:
              count    100.000000
mean     440.809998
std       59.781536
min      303.000000
25%      400.500000
50%      439.000000
75%      473.000000
max      620.000000
Name: 0, dtype: float64
             
 band 3:
              count    100.000000
mean     265.130005
std       66.175102
min      176.000000
25%      225.750000
50%      244.000000
75%      296.250000
max      499.000000
Name: 0, dtype: float64
             
 band 4:
              count     100.000000
mean     2444.080078
std       258.430206
min      1748.000000
25%      2300.500000
50%      2392.000000
75%      2635.500000
max      3060.000000
Name: 0, dtype: float64
             
 band 5:
              count     100.000000
mean     2781.929932
std       206.534378
min      2318.000000
2

In [41]:
r.shape

(9, 100, 100)

In [148]:
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
data = np.load(path) 
train_examples = data['x_train']
train_labels = data['y_train']
test_examples = data['x_test']
test_labels = data['y_test']

In [158]:
train_examples[0].shape
# train_labels.shape

(28, 28)

In [100]:
image_path = "./PetImages/kagglecatsanddogs_3367a/PetImages/Cat/0.jpg"
image_path_2 = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/yes/Habitation/100/6/6_300_1100.tif"

# PetImages/kagglecatsanddogs_3367a

def convert_rast(input_dir,output_dir,band_list=[6,7,8],data_type="uint8"):
    
    input_dir = add_trailing_slash(input_dir)
    output_dir = add_trailing_slash(output_dir)
    
    create_dir(output_dir)
    
    file_list = glob(input_dir + "/*/*.tif")
   
    
    for image_path in file_list:
        new_dir = output_dir + image_path.split("/")[-2]
        new_dir = add_trailing_slash(new_dir)
        create_dir(new_dir)
        
        file_path_append = ("/").join(image_path.split("/")[-2:]) 
        
        with rio.open(image_path, 'r') as r:
            rast = r.read(band_list)
            rast = rast.astype(np.uint8)
            r_meta = r.meta
            r_meta["count"] = len(band_list)
            r_meta["dtype"] = data_type


        with rio.open(output_dir + file_path_append, 'w', **r_meta) as outf:
            outf.write(rast)
        

In [ ]:
path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
    train_examples = data['x_train']
    train_labels = data['y_train']
    test_examples = data['x_test']
    test_labels = data['y_test']

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

In [104]:
# np.unique(rast_2[8], return_counts=True)

# np.min(rast_2[8])
# np.max(rast_2[8])

# Keras TIF Directory Preprocessing Solution - Will not work for more than 3 or 4 bands or TIFF images!

In [134]:
batch_size = 32
img_height = 180
img_width = 180

In [132]:
data_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/test/blah_jpg"

In [133]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  labels="inferred",
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 0 files belonging to 2 classes.
Using 0 files for training.


TypeError: Input 'filename' of 'ReadFile' Op has type float32 that does not match expected type of string.

In [129]:
train_ds

<BatchDataset shapes: ((None, 180, 180, 3), (None,)), types: (tf.float32, tf.int32)>

In [36]:
def rename_dir(data_dir):
    dir_list = glob(f'{data_dir}/*/*.tif')
    dir_list_new = [f'{"/".join(i.split("/")[:-1])}/abc_{i.split("/")[-1]}' for i in dir_list] 
    for i in range(len(dir_list)):
        os.rename(dir_list[i],dir_list_new[i])
        
#     print(dir_list)

In [37]:
data_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/test/100"

rename_dir(data_dir)

In [24]:
import io_ops

dir_list = glob(f'{data_dir}/*/*.tif')

# [f'{"/".join(i.split("/")[:-1])}/{i.split("/")[-2]}_{i.split("/")[-1]}' for i in dir_list]

img = io_ops.read_file(path)

ModuleNotFoundError: No module named 'io_ops'

In [28]:
tf.io.read_file(dir_list[0])

<tf.Tensor: shape=(), dtype=string, numpy=b'II*\x00\x08\x00\x00\x00\x12\x00\x00\x01\x03\x00\x01\x00\x00\x00d\x00\x00\x00\x01\x01\x03\x00\x01\x00\x00\x00d\x00\x00\x00\x02\x01\x03\x00\t\x00\x00\x00\xe6\x00\x00\x00\x03\x01\x03\x00\x01\x00\x00\x00\x01\x00\x00\x00\x06\x01\x03\x00\x01\x00\x00\x00\x01\x00\x00\x00\x11\x01\x04\x002\x00\x00\x00\\\x01\x00\x00\x15\x01\x03\x00\x01\x00\x00\x00\t\x00\x00\x00\x16\x01\x03\x00\x01\x00\x00\x00\x02\x00\x00\x00\x17\x01\x03\x002\x00\x00\x00\xf8\x00\x00\x00\x1c\x01\x03\x00\x01\x00\x00\x00\x01\x00\x00\x00R\x01\x03\x00\x08\x00\x00\x00$\x02\x00\x00S\x01\x03\x00\t\x00\x00\x004\x02\x00\x00\x0e\x83\x0c\x00\x03\x00\x00\x00\xce\x04\x00\x00\x82\x84\x0c\x00\x06\x00\x00\x00\xe6\x04\x00\x00\xaf\x87\x03\x00 \x00\x00\x00\x16\x05\x00\x00\xb0\x87\x0c\x00\x02\x00\x00\x00V\x05\x00\x00\xb1\x87\x02\x00\x08\x00\x00\x00f\x05\x00\x00\x80\xa4\x02\x00\x87\x02\x00\x00F\x02\x00\x00\x00\x00\x00\x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x1c \x1c \x1c \x1c \x1c \x1c \x1c \x1c \x1

# From Starter

In [64]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [65]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0  4403k      0  0:03:02  0:03:02 --:--:-- 4853k0:02:41 5952k 4642k  0  0:03:06  0:02:35  0:00:31 1768k7  0:02:38  0:00:29 3233k


In [78]:
!unzip -q kagglecatsanddogs_3367a.zip
!ls

replace readme[1].txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
LICENSE                     kagglecatsanddogs_3367a.zip
MSR-LA - 3467.docx          readme[1].txt
PetImages                   starter-explore
README.md


In [72]:
pet_dir = './PetImages'

glob(f'{pet_dir}/*/*')[0:10]

['./PetImages/Cat/9733.jpg',
 './PetImages/Cat/63.jpg',
 './PetImages/Cat/6400.jpg',
 './PetImages/Cat/823.jpg',
 './PetImages/Cat/4217.jpg',
 './PetImages/Cat/3578.jpg',
 './PetImages/Cat/10321.jpg',
 './PetImages/Cat/2666.jpg',
 './PetImages/Cat/5109.jpg',
 './PetImages/Cat/11981.jpg']

In [74]:
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)


Deleted 1000 images


In [130]:
image_size = (180, 180)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "./PetImages/kagglecatsanddogs_3367a/PetImages/",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     "PetImages",
#     validation_split=0.2,
#     subset="validation",
#     seed=1337,
#     image_size=image_size,
#     batch_size=batch_size,
# )

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [131]:
train_ds

<BatchDataset shapes: ((None, 180, 180, 3), (None,)), types: (tf.float32, tf.int32)>